# Usando *Machine Learning* para prever valores de casas em *King County, WA*

<p align="center">
    <img width=750, src="https://github.com/DanOliveira-DS/Datasets_for_Practicing_Applied_Machine_Learning/blob/master/imgs/king_county.jpg?raw=true">
</p>




# Contexto do Desafio
A House Rocket é uma plataforma digital que tem como modelo de negócio, a compra e a venda de imóveis usando tecnologia.

Você é um Data Scientist contrato pela empresa para ajudar a encontrar as melhores oportunidades de negócio no mercado de imóveis. O CEO da House Rocket gostaria de maximizar a receita da empresa encontrando boas oportunidades de negócio.

Sua principal estratégia é comprar boas casas em ótimas localizações com preços baixos e depois revendê-las posteriormente à preços mais altos. Quanto maior a diferença entre a compra e a venda, maior o lucro da empresa e portanto maior sua receita.

Entretanto, as casas possuem muitos atributos que as tornam mais ou menos atrativas aos compradores e vendedores e a localização e o período do ano também podem influenciar os preços.

Portanto, seu trabalho como Data Scientist é responder as seguinte perguntas:

* Quais casas o CEO da House Rocket deveria comprar e por qual preço de compra?
* Uma vez a casa em posse da empresa, qual o melhor momento para vendê-las e qual seria o preço da venda?
* A House Rocket deveria fazer uma reforma para aumentar o preço da venda? Quais seriam as sugestões de mudanças? 
* Qual o incremento no preço dado por cada opção de reforma?

Para avaliar a performance do nosso modelo vamos utilizar:

* Raiz quadrada do erro-médio - 
$RMSE = \sqrt{\frac{1}{n}\Sigma_{i=1}^{n}{\Big(\frac{d_i -f_i}{\sigma_i}\Big)^2}}$

# Descrição das variáveis



# Roteiro Sugerido para a Resolução.
Esse é o roteiro de resolução:

1. Identifique a causa raíz.
    * Porque o CEO fez essas perguntas? Se você fosse ele, porque você perguntaria isso? Quer aumentar receita? A empresa está indo bem?
    * Anote essas causas.
2. Colete os dados ( Os dados estão no link acima )
3. Aplique uma limpeza nos dados.
    * Entenda as variáveis disponíveis, possíveis valores faltantes, faça uma estatística descritiva para entender as características dos dados.
4. Levante Hipóteses sobre o Comportamento do Negócio.
    * Casas com garagens são mais caras? Porque?
    * Casas com muitos quartos são mais caras? Porque? A partir de quantos quartos o preço aumenta? Qual o incremento de preço por cada quarto adicionado?
    * As casas mais caras estão no centro? Qual a região? Existe alguma coisa na região que tem correlação com valor de venda da casa? Shoppings? Montanhas? Pessoas Famosas?
5. Faça uma ótima Análise Exploratória de Dados.
    * Quais hipóteses são falsas e quais são verdadeiras?
    * Quais as correlações entre as variáveis e a variável resposta?
6. Escreve os Insights que você encontro.
7. Escreve possíveis soluções para o problema do CEO.